In [ ]:
import numpy as np
import pandas as pd
import xarray

In [ ]:
gfrac = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_new.nc", engine="netcdf4")
gbiomass_5min = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS_new.nc", engine="netcdf4")

trans_glct1st_netcdf = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans1st.nc", engine="netcdf4")
glct_trans = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans.nc", engine="netcdf4")
gfrac_ngfbfc = [element.strip() for element in gfrac.coords['NGFBFC'].data.astype('str').tolist()]

In [ ]:
# tabel 1

selisih_gfracarea_table1 = np.zeros((10, 2160, 4320), dtype="float32")

max_year = 10
for time in range(max_year):
    if time < max_year - 1: # 0 = 1975
        selisih_gfracarea_table1[time+1] = np.where(trans_glct1st_netcdf['GLCT_1st'].isel(time=time).values == 'natveg_to_agri',
                                                            np.where((trans_glct1st_netcdf['GLCT_1st'].isel(time=time+1).values == 'agri_to_agri') |
                                                                    (trans_glct1st_netcdf['GLCT_1st'].isel(time=time+1).values == 'agri_to_natveg'), 1, 0), 0)

In [ ]:
table1_netcdf = xarray.Dataset({
        "gfrac_selisih":(["time", "latitude", "longitude"], selisih_gfracarea_table1)
    },
    coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": glct_trans.coords["latitude"].to_numpy(),
        "longitude": glct_trans.coords["longitude"].to_numpy()
    })
table1_netcdf

In [ ]:
# tabel 2

selisih_gfracarea_table2 = np.zeros((11, 2160, 4320), dtype="float32")

max_year = 10
for i in range(max_year): #1970
    selisih_gfracarea_table2[i+1] = np.where(glct_trans['GLCT_trans'].isel(time=i).values == "agri",
     np.where(gbiomass_5min['GBIOMASS_new'].isel(time=i+1).values < 400,
     np.where(gbiomass_5min['GBIOMASS_new'].isel(time=i).values > 400, 1, 0), 0), 0)

In [ ]:
table2_netcdf = xarray.Dataset({
        "gfrac_selisih":(["time", "latitude", "longitude"], selisih_gfracarea_table2)
    },
    coords={
        "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
        "latitude": glct_trans.coords["latitude"].to_numpy(),
        "longitude": glct_trans.coords["longitude"].to_numpy(),
    })
table2_netcdf

In [ ]:
data = table1_netcdf + table2_netcdf.isel(time=slice(1, 11))
data

In [ ]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]

data.sel(latitude=[0], longitude=[0]).to_dataframe()